In [1]:
import h5py as h5
#from h5glance import H5Glance
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import gvar as gv
import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path
import importlib

In [2]:
#import local files
#sys.path.append("../")
import gmo_fitter as gmo 
import i_o 
import gmo_fit_analysis as fa
import load_data_priors as ld
import mass_relations as ma
matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
from input_files import a12m180L
# from input_files import a12m180S
# from input_files import a12m220
# from input_files import a12m220L
# from input_files import a12m220S
# from input_files import a12m220XL
# from input_files import a12m220_ms
# from input_files import a12m260
# from input_files import a12m310
# from input_files import a12m310L
# from input_files import a15m130
# from input_files import a15m310


p_dict = a12m180L.p_dict
abbr = p_dict['abbr']

In [4]:
import platform
if platform.system() == 'Darwin':
    file = '/Users/grantdb/lqcd/data/c51_2pt_octet_decuplet.h5'
else:
    file = '/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5'
with h5.File(file,"r") as f:
    ensembles = {}
    ensembles = list(f.keys())
    # print(ensembles)

In [5]:
# hacky way to initialize all raw corr data 
importlib.reload(ld)
nucleon_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='proton')
lam_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='lambda_z')
xi_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='xi_z')
sigma_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='sigma_p')
piplus_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='piplus')
kplus_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='kplus')
delta_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='delta_pp')
gmo_ratio_raw = ld.G_gmo(file,p_dict['abbr'],log=False)
# eff_mass = ma.gmo_eff_mass(gmo_out=gmo_ratio_raw,dt=1)
# gmo_ratio_raw
# nucleon_corr


/home/gmoney/lqcd/gmo_relation/load_data_priors.py:55: RuntimeWarning: invalid value encountered in power
  * np.power(temp[('proton', smr)], -2/3)


In [6]:
prior_nucl = {}
prior = {}
states=p_dict['gmo_states_all']
newlist = [x for x in states]
for x in newlist:
    path = os.path.normpath("./priors/{0}/{1}/prior_nucl.csv".format(p_dict['abbr'],x))
    df = pd.read_csv(path, index_col=0).to_dict()
    for key in list(df.keys()):
        length = int(np.sqrt(len(list(df[key].values()))))
        prior_nucl[key] = list(df[key].values())[:length]
        # prior_nucl['gmo_E'] = list([np.repeat(gv.gvar('0.0030(27)'),8)])
    prior = gv.gvar(prior_nucl)
# prior_gmo = {}
# prior['gmo_E'] = gv.gvar('0.0030(27)') #this shouldnt be priored like this? #

In [7]:
model_type = 'pi'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior, 
nucleon_corr_data=None,lam_corr_data=None, xi_corr_data=None,sigma_corr_data=None,delta_corr_data=None,
piplus_corr_data=piplus_corr,kplus_corr_data=None,model_type=model_type)
print(gmo_.get_fit().format(maxline=True))
fit_out = gmo_.get_fit()
# gmo_.plot_effective_mass()
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='meson')

Least Square Fit:
  chi2/dof [dof] = 0.84 [50]    Q = 0.78    logGBF = 575.69

Parameters:
        gmo_E0 0           0.3 (2.5)        [    0.3 (2.5) ]  
          z_PS 0   -4e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1   -3e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
          z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
    sigma_z_PS 0        0.0(3.3)e-06        [ 0.0(3.3)e-06 ]  
               1        0.0(3.3)e-06        [ 0.0(3.3)e-06 ]  
    sigma_z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
      lam_z_PS 0        0.0(3.3)e-06        [ 0.0(3.3)e-06 ]  
               1        0.0(3.3)e-06        [ 0.0(3.3)e-06 ]  
      lam_z_SS 0        4.4(4.4)e-06        [ 4.4(4.4)e-06 ]  
               1        4.4(4.4)e-06        [ 4.4(4.4)e-06 ]  
   proton_z_PS 0   -6e-21 +- 3.3e-05        [ 0.0(3.3)e-05 ]  
               1    1e-21 +

'fit_results/a12m180L/meson_fit_params'

In [8]:
model_type = 'kplus'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior, 
nucleon_corr_data=None,lam_corr_data=None, xi_corr_data=None,sigma_corr_data=None,delta_corr_data=None,
piplus_corr_data=None,kplus_corr_data=kplus_corr,model_type=model_type)
print(gmo_.get_fit().format(maxline=True))
fit_out = gmo_.get_fit()
# gmo_.plot_effective_mass()
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='meson')

Least Square Fit:
  chi2/dof [dof] = 0.94 [64]    Q = 0.6    logGBF = 1071.1

Parameters:
        gmo_E0 0           0.3 (2.5)        [    0.3 (2.5) ]  
          z_PS 0   -9e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1    5e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
          z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
    sigma_z_PS 0        0.0(3.3)e-06        [ 0.0(3.3)e-06 ]  
               1    9e-42 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
    sigma_z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
      lam_z_PS 0   -8e-42 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1   -1e-41 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
      lam_z_SS 0        4.4(4.4)e-06        [ 4.4(4.4)e-06 ]  
               1        4.4(4.4)e-06        [ 4.4(4.4)e-06 ]  
   proton_z_PS 0   -8e-22 +- 3.3e-05        [ 0.0(3.3)e-05 ]  
               1   -4e-21 +-

'fit_results/a12m180L/meson_fit_params'

In [9]:
model_type='pi'


gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=25, model_type='pi', fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

UnboundLocalError: local variable 'nucleon_corr_gv' referenced before assignment

In [ ]:
model_type='kplus'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=25, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**

In [ ]:
#pickle gvar priors/posteriors from fit using gvar dump
fit_out = fit_ensemble.get_fit()
fit_dump = {}
fit_dump['prior'] = fit_out.prior
fit_dump['p'] = fit_out.p
fit_dump['logGBF'] = fit_out.logGBF
fit_dump['Q'] = fit_out.Q
print(fit_dump)
out_path = 'fit_results/{1}/'.format(p_dict['fit_state'],p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
gv.dump(fit_dump, out_path+'/{0}_{1}'.format(p_dict['fit_state'],p_dict['abbr']))

((7, 15, 2),)
{'prior': BufferDict([('wf_dir', array([0.0(3.3)e-07, 0.0(3.3)e-07], dtype=object)), ('wf_smr', array([4.4(4.4)e-06, 4.4(4.4)e-06], dtype=object)), ('z_PS', array([0.0(4.3)e-06, 0.0(4.3)e-06], dtype=object)), ('z_SS', array([9.4(9.4)e-07, 9.4(9.4)e-07], dtype=object)), ('E0', 0.78(22)), ('log(dE)', array([-0.87(76)], dtype=object))]), 'p': BufferDict([('wf_dir', array([2e-23 +- 3.3e-07, 1e-23 +- 3.3e-07], dtype=object)), ('wf_smr', array([4.4(4.4)e-06, 4.4(4.4)e-06], dtype=object)), ('z_PS', array([3.539(52)e-06, 5.71(77)e-06], dtype=object)), ('z_SS', array([9.20(12)e-07, 8.21(92)e-07], dtype=object)), ('E0', 0.7810(11)), ('log(dE)', array([-0.834(68)], dtype=object))]), 'logGBF': 416.36021163737547, 'Q': 0.9175269467842375}


'fit_results/a12m180L//xi_z_a12m180L'

: 

: 

: 

: 

: 

: 

: 

In [ ]:
#comment out plot commands in fit_manager for ga, gv for hp ens
if p_dict['make_plots']:
    plots = fit_ensemble.make_plots()
    output_dir = 'ga_fit_results/{0}/{1}_{0}.pdf'.format(p_dict['abbr'],p_dict['part'])
    output_pdf = PdfPages(output_dir)
    for plot in plots:
        if plot is not None:
            output_pdf.savefig(plot)
    if p_dict['show_many_states']:
        output_pdf.savefig(fit_ensemble.plot_stability(model_type='corr', n_states_array=[1, 2, 3, 4]))
    output_pdf.close()

<IPython.core.display.Javascript object>

((7, 15, 2),)
0 PS


/home/gmoney/lqcd/gmo_mq/fit_manager.py:122: RuntimeWarning: invalid value encountered in log
  return {key : 1/dt * np.log(nucleon_corr_gv[key] / np.roll(nucleon_corr_gv[key], -1))


<IPython.core.display.Javascript object>

1 SS
((7, 15, 2),)


...

: 

: 

: 

: 

: 

## bootstrapping ##

bs resampling and redo fits 

keep track of seed 

gauge average bc approximate integral that way -> physical point

data from configs, resample gauge configs, average them to get result of integral

rand sample m times (gauge field configs) to compute bs sample 

array n x m, average over m (n number of resamples)

each resample n correspond to an integral to be computed by avg over m configurations

https://lsqfit.readthedocs.io/en/latest/lsqfit.html
1) make a large number of “bootstrap copies” of the original input data and prior that differ from each other by random amounts characteristic of the underlying randomness in the original data

2) repeat the entire fit analysis for each bootstrap copy of the data, extracting fit results from each 

3) use the variation of the fit results from bootstrap copy to bootstrap copy to determine an approximate probability distribution (possibly non-gaussian) for the fit parameters and/or functions of them: the results from each bootstrap fit are samples from that distribution.

In [ ]:
def bs_to_gvar(data, bs_list):
    bs_M = data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
    #bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
    
    temp_dict = {}
    for key in data.keys():
        temp = data[key][bs_list[0:]
        temp_dict[key] = np.mean(temp, axis=0).real
        
    for k in range(1, bs_N):
        for key in data.keys():
            temp = data[key][bs_list[k:]
            temp_dict[key] = np.vstack((temp_dict[key], np.mean(temp, axis=0).real))
    
    output = {}
    #return temp_dict
    for key in data.keys():
        mean = np.mean(data[key], axis=0)
        unc = np.cov(temp_dict[key], rowvar=False)
        output[key] = gv.gvar(mean,unc)
    return output

nucleon_bs_list = bs_to_gvar(nucleon_corr_data, 500)
axial_bs_list   = bs_to_gvar(axial_fh_num_data,500)
vector_bs_list  = bs_to_gvar(vector_fh_num_data,500)
#nucleon_bs_list


SyntaxError: invalid syntax (<ipython-input-515-3bdba660a7e5>, line 8)

: 

: 

: 

: 

: 

: 

: 

In [ ]:
#b = gv.dataset.bootstrap_iter(nucleon_bs_list)
#print(next(b))b

: 

: 

: 

: 

: 

: 

: 

In [ ]:
bs_list_path = os.path.realpath('./c51_mdwf_hisq_spec.h5')
#H5Glance(bs_list_path)

: 

: 

: 

: 

: 

: 

: 

In [ ]:
with h5.File(bs_list_path, "r") as f: 
    bs = f["/"+ p_dict['abbr']+"/bs_lst"]
    bs_list = np.array(bs)
    print(bs_list)

OSError: Unable to open file (unable to open file: name = '/Users/grantdb/hyperon_ga/c51_mdwf_hisq_spec.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

: 

: 

: 

: 

: 

: 

: 

In [ ]:
#bs_list[1,:]

: 

: 

: 

: 

: 

: 

: 

In [ ]:
print(len(bs_list))
# bs_M = nucleon_corr_data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
bs_N = 1000 # In real-world cases, this would probably be much larger
# bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
# bs_list
#bs_list[:,:]

NameError: name 'bs_list' is not defined

: 

: 

: 

: 

: 

: 

: 

In [ ]:
n_time_slices = nucleon_corr_data['smr'].shape[1] # again, could use 'ps' instead
bs_correlators_gv = []
for k in range(bs_N):
    temp = ({key : nucleon_corr_data[key][bs_list[k,:], :]
                           for key in nucleon_corr_data.keys()})
    
    # convert each bootstrap resample into a correlator with uncertainty
    # by using gv.dataset.avg_data (same function we used to make original data for fit)
    bs_correlators_gv.append(gv.dataset.avg_data(temp))
#bs_correlators_gv

: 

: 

: 

: 

: 

: 

: 

In [ ]:
fit_ensemble_bs = fm.fit_ensemble(t_range=p_dict['t_range'],
                               n_states=p_dict['n_states'], prior=prior, 
                               nucleon_corr_data=bs_correlators_gv, 
                               axial_fh_num_data=axial_fh_num_data,
                               vector_fh_num_data=vector_fh_num_data)
print(fit_ensemble_bs)

: 

: 

: 

: 

: 

: 

: 

In [ ]:
# Make bs_N fits, one for each of the resampled correlators
fit_keys = fit_out.p.keys()
output = {key : [] for key in fit_keys}

for j, bsfit in enumerate(fit_out.bootstrapped_fit_iter(datalist=bs_correlators_gv)):
    for key in fit_keys:
        # Save the best estimate for the central value 
        # of each parameter of each fit
        p = bsfit.pmean[key]
        output[key].append(p)

# # print results -- should be similar to previous results
table = gv.dataset.avg_data(output, bstrap=True)
print(gv.tabulate(table))

: 

: 

: 

: 

: 

: 

: 

In [ ]:
#print(gv.evalcorr(table[key][:2])[1,0])
# measure of how well multi-dimensional Gaussian distributions(regular fit and bootstrapped fit) agree with each other 
# do their means agree within errors for corresponding elements. 
# note they share the same keys, so just loop over those 
# only parts of fit params that overlap are used 
for key in fit_keys:
    chi2 = gv.chi2(table[key],fit_out.p[key])
gv.fmt_chi2(chi2)

: 

: 

: 

: 

: 

: 

: 